# Генерация синтетических данных для имитации результата base pipeline
В этом ноутбуке создаются случайные данные за 10 месяцев, аналогичные формату результата функции `run_base_pipeline`.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import random
import os

# Для воспроизводимости
np.random.seed(42)
random.seed(42)

# Диапазон дат: с 2024-10-31 по 2025-07-31 (10 месяцев, последний день месяца)
N_MONTHS = 10
START_DATE = datetime(2024, 10, 1)
DATE_RANGE = [(START_DATE + relativedelta(months=i) + pd.offsets.MonthEnd(0)).date() for i in range(N_MONTHS)]

ARTICLES = ['Статья_1']
ITEM_ID = 'INDIVIDUAL'
FACTOR = 'INDIVIDUAL'

MODEL_COLS = [
    'NaiveModel', 'SeasonalNaiveModel', 'AverageModel', 'SeasonalAverageModel', 'ZeroModel',
    'AutoARIMAModel', 'AutoETSModel', 'ThetaModel', 'CrostonModel', 'NPTSModel', 'DeepARModel',
    'PatchTSTModel', 'TemporalFusionTransformerModel', 'TiDEModel', 'DirectTabularModel', 'RecursiveTabularModel',
    'Chronos_base', 'Chronos_small', 'Chronos_small_finetuned',
    'predict_svm9', 'predict_linreg9_no_bias', 'predict_RFR'
    ]

# Генерация базовых временных рядов
rows = []
for article in ARTICLES:
    base = np.linspace(100, 200, N_MONTHS) + np.random.normal(0, 10, N_MONTHS)
    for i, last_day in enumerate(DATE_RANGE):
        row = {
            'date': last_day,
            'Fact': base[i],
            'Статья': article,
            'item_id': ITEM_ID,
            'factor': FACTOR
        }
        rows.append(row)
df = pd.DataFrame(rows)

# Генерация предсказаний для всех моделей
for col in MODEL_COLS:
    if col == 'NaiveModel':
        df[col] = df['Fact'] + np.random.normal(0, 8, N_MONTHS)
    elif col == 'SeasonalNaiveModel':
        df[col] = df['Fact'] + np.random.normal(0, 7, N_MONTHS)
    elif col == 'AverageModel':
        df[col] = df['Fact'] + np.random.normal(0, 6, N_MONTHS)
    elif col == 'SeasonalAverageModel':
        df[col] = df['Fact'] + np.random.normal(0, 6, N_MONTHS)
    elif col == 'ZeroModel':
        df[col] = np.zeros(N_MONTHS)
    elif col == 'AutoARIMAModel':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 6, N_MONTHS)
    elif col == 'AutoETSModel':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 5, N_MONTHS)
    elif col == 'ThetaModel':
        df[col] = df['Fact'] * np.random.uniform(0.95, 1.05, N_MONTHS) + np.random.normal(0, 7, N_MONTHS)
    elif col == 'CrostonModel':
        df[col] = df['Fact'] * np.random.uniform(0.94, 1.06, N_MONTHS) + np.random.normal(0, 7, N_MONTHS)
    elif col == 'NPTSModel':
        df[col] = df['Fact'] * np.random.uniform(0.93, 1.07, N_MONTHS) + np.random.normal(0, 8, N_MONTHS)
    elif col == 'DeepARModel':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 5, N_MONTHS)
    elif col == 'PatchTSTModel':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'TemporalFusionTransformerModel':
        df[col] = df['Fact'] * np.random.uniform(0.95, 1.05, N_MONTHS) + np.random.normal(0, 6, N_MONTHS)
    elif col == 'TiDEModel':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 5, N_MONTHS)
    elif col == 'DirectTabularModel':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'RecursiveTabularModel':
        df[col] = df['Fact'] * np.random.uniform(0.95, 1.05, N_MONTHS) + np.random.normal(0, 6, N_MONTHS)
    elif col == 'Chronos_base':
        df[col] = df['Fact'] * np.random.uniform(0.98, 1.02, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'Chronos_small':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'Chronos_small_finetuned':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'predict_svm9':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 5, N_MONTHS)
    elif col == 'predict_linreg9_no_bias':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'predict_RFR':
        df[col] = df['Fact'] * np.random.uniform(0.95, 1.05, N_MONTHS) + np.random.normal(0, 6, N_MONTHS)

# Для каждой модели считаем разницу и отклонение
for col in MODEL_COLS:
    df[f'{col}_разница'] = df[col] - df['Fact']
    df[f'{col}_отклонение_%'] = 100 * (df[col] - df['Fact']) / df['Fact']

df = df.sort_values('date').reset_index(drop=True)

# Генерация коэффициентов и информации об ансамблях
coeffs_with_intercept = pd.DataFrame({
    'date': [DATE_RANGE[-1]],
    'Статья': [ARTICLES[0]],
    'predict_naive': [np.random.uniform(0.2, 0.4)],
    'predict_TS_ML': [np.random.uniform(0.3, 0.5)],
    'intercept': [np.random.uniform(10, 20)]
})
coeffs_no_intercept = pd.DataFrame({
    'date': [DATE_RANGE[-1]],
    'Статья': [ARTICLES[0]],
    'predict_naive': [np.random.uniform(0.3, 0.6)],
    'predict_TS_ML': [np.random.uniform(0.3, 0.6)]
})
ensemble_info = pd.DataFrame({
    'date': [DATE_RANGE[-1]],
    'Статья': [ARTICLES[0]],
    'model': ['Ensemble_TS_ML'],
    'weight': [1.0],
    'metric': ['MAE'],
    'score': [np.random.uniform(5, 15)]
})

# Формирование файла для BASE pipeline
base_cols = ['date', 'Fact'] + MODEL_COLS + ['Статья', 'item_id', 'factor']
df_base = df[base_cols].copy()
df_base = df_base.rename(columns={'date': 'Дата'})
base_cols = ['Дата', 'Fact'] + MODEL_COLS + ['Статья', 'item_id', 'factor']
df_base = df_base[base_cols]

# Переименование 'date' в 'Дата' во всех листах
coeffs_with_intercept = coeffs_with_intercept.rename(columns={'date': 'Дата'})
coeffs_no_intercept = coeffs_no_intercept.rename(columns={'date': 'Дата'})
ensemble_info = ensemble_info.rename(columns={'date': 'Дата'})

# Сохранение результатов
os.makedirs('results', exist_ok=True)
with pd.ExcelWriter('results/synthetic_base_pipeline_BASE.xlsx') as writer:
    df_base.to_excel(writer, sheet_name='data', index=False)
    coeffs_with_intercept.to_excel(writer, sheet_name='coeffs_with_intercept', index=False)
    coeffs_no_intercept.to_excel(writer, sheet_name='coeffs_no_intercept', index=False)
    ensemble_info.to_excel(writer, sheet_name='TimeSeries_ensemble_models_info', index=False)
print('Файл сохранён: results/synthetic_base_pipeline_BASE.xlsx (с "Дата" во всех листах)')

Файл сохранён: results/synthetic_base_pipeline_BASE.xlsx (с "Дата" во всех листах)


In [2]:
# Установка random seed для воспроизводимости
np.random.seed(42)
random.seed(42)

In [3]:
# Параметры генерации данных
N_MONTHS = 10
END_DATE = datetime.today().replace(day=1)
DATE_RANGE = [END_DATE - relativedelta(months=i) for i in range(N_MONTHS-1, -1, -1)]

ARTICLES = ['Статья_1']  # Можно добавить больше статей
FACTOR = 'FACTOR_1'

MODEL_COLS = [
    'predict_naive',
    'predict_TS_ML',
    'predict_autoARIMA',
    'predict_TFT',
    'predict_PatchTST',
    'predict_Chronos_base',
    'predict_svm9',
    'predict_linreg9_no_bias',
    'predict_RFR'
]

In [4]:
# --- Явно задаём диапазон дат для покрытия нужных месяцев (например, с июля 2024 по июнь 2025) ---
N_MONTHS = 12
START_DATE = datetime(2024, 7, 1)
DATE_RANGE = [START_DATE + relativedelta(months=i) for i in range(N_MONTHS)]
print('Диапазон дат:', [d.strftime('%Y-%m-%d') for d in DATE_RANGE])

Диапазон дат: ['2024-07-01', '2024-08-01', '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01', '2025-01-01', '2025-02-01', '2025-03-01', '2025-04-01', '2025-05-01', '2025-06-01']


In [5]:
# --- Увеличиваем диапазон дат до 24 месяцев с февраля 2024 ---
N_MONTHS = 24
START_DATE = datetime(2024, 2, 1)
DATE_RANGE = [START_DATE + relativedelta(months=i) for i in range(N_MONTHS)]
print('Диапазон дат:', [d.strftime('%Y-%m-%d') for d in DATE_RANGE])

Диапазон дат: ['2024-02-01', '2024-03-01', '2024-04-01', '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01', '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01', '2025-01-01', '2025-02-01', '2025-03-01', '2025-04-01', '2025-05-01', '2025-06-01', '2025-07-01', '2025-08-01', '2025-09-01', '2025-10-01', '2025-11-01', '2025-12-01', '2026-01-01']


In [6]:
# Генерация базовых временных рядов
rows = []
for article in ARTICLES:
    base = np.linspace(100, 200, N_MONTHS) + np.random.normal(0, 10, N_MONTHS)
    for i, date in enumerate(DATE_RANGE):
        row = {
            'date': date,
            'Fact': base[i],
            'Статья': article
        }
        rows.append(row)
df = pd.DataFrame(rows)

In [7]:
# --- Приводим столбец 'date' к последнему дню месяца для совместимости с pipeline ---
df['date'] = pd.to_datetime(df['date']) + pd.offsets.MonthEnd(0)
print('Первые даты после преобразования:', df['date'].head())

Первые даты после преобразования: 0   2024-02-29
1   2024-03-31
2   2024-04-30
3   2024-05-31
4   2024-06-30
Name: date, dtype: datetime64[ns]


In [8]:
# Генерация предсказаний для отдельных моделей
for col in MODEL_COLS:
    if col == 'predict_naive':
        df[col] = df['Fact'] + np.random.normal(0, 8, N_MONTHS)
    elif col == 'predict_TS_ML':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 5, N_MONTHS)
    elif col == 'predict_autoARIMA':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 6, N_MONTHS)
    elif col == 'predict_TFT':
        df[col] = df['Fact'] * np.random.uniform(0.95, 1.05, N_MONTHS) + np.random.normal(0, 7, N_MONTHS)
    elif col == 'predict_PatchTST':
        df[col] = df['Fact'] * np.random.uniform(0.94, 1.06, N_MONTHS) + np.random.normal(0, 7, N_MONTHS)
    elif col == 'predict_Chronos_base':
        df[col] = df['Fact'] * np.random.uniform(0.93, 1.07, N_MONTHS) + np.random.normal(0, 8, N_MONTHS)
    elif col == 'predict_svm9':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 5, N_MONTHS)
    elif col == 'predict_linreg9_no_bias':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'predict_RFR':
        df[col] = df['Fact'] * np.random.uniform(0.95, 1.05, N_MONTHS) + np.random.normal(0, 6, N_MONTHS)

In [9]:
# Генерация коэффициентов для линейных моделей и информации об ансамблях
coeffs_with_intercept = pd.DataFrame({
    'date': [DATE_RANGE[-1]],
    'Статья': [ARTICLES[0]],
    'predict_naive': [np.random.uniform(0.2, 0.4)],
    'predict_TS_ML': [np.random.uniform(0.3, 0.5)],
    'intercept': [np.random.uniform(10, 20)]
})

coeffs_no_intercept = pd.DataFrame({
    'date': [DATE_RANGE[-1]],
    'Статья': [ARTICLES[0]],
    'predict_naive': [np.random.uniform(0.3, 0.6)],
    'predict_TS_ML': [np.random.uniform(0.3, 0.6)]
})

ensemble_info = pd.DataFrame({
    'date': [DATE_RANGE[-1]],
    'Статья': [ARTICLES[0]],
    'model': ['Ensemble_TS_ML'],
    'weight': [1.0],
    'metric': ['MAE'],
    'score': [np.random.uniform(5, 15)]
})

In [10]:
# Расчет ошибок (разница и отклонение в процентах)
for col in MODEL_COLS:
    df[f'{col}_разница'] = df[col] - df['Fact']
    df[f'{col}_отклонение_%'] = 100 * (df[col] - df['Fact']) / df['Fact']

df = df.sort_values('date').reset_index(drop=True)
df['date'] = df['date'].dt.date  # Для совместимости с Excel

In [11]:
# Сохранение результатов в Excel-файл
os.makedirs('results', exist_ok=True)
with pd.ExcelWriter('results/synthetic_base_pipeline.xlsx') as writer:
    df.to_excel(writer, sheet_name='data', index=False)
    coeffs_with_intercept.to_excel(writer, sheet_name='coeffs_with_intercept', index=False)
    coeffs_no_intercept.to_excel(writer, sheet_name='coeffs_no_intercept', index=False)
    ensemble_info.to_excel(writer, sheet_name='TimeSeries_ensemble_models_info', index=False)
print('Файл сохранён: results/synthetic_base_pipeline.xlsx')

Файл сохранён: results/synthetic_base_pipeline.xlsx


In [12]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import random
import os

# Для воспроизводимости
np.random.seed(42)
random.seed(42)

# Параметры генерации
N_MONTHS = 10
END_DATE = datetime.today().replace(day=1)
DATE_RANGE = [END_DATE - relativedelta(months=i) for i in range(N_MONTHS-1, -1, -1)]

ARTICLES = ['Статья_1']
ITEM_ID = 'INDIVIDUAL'
FACTOR = 'INDIVIDUAL'

# Все модели из config_refined.yaml
MODEL_COLS = [
    'NaiveModel', 'SeasonalNaiveModel', 'AverageModel', 'SeasonalAverageModel', 'ZeroModel',
    'AutoARIMAModel', 'AutoETSModel', 'ThetaModel', 'CrostonModel', 'NPTSModel', 'DeepARModel',
    'PatchTSTModel', 'TemporalFusionTransformerModel', 'TiDEModel', 'DirectTabularModel', 'RecursiveTabularModel',
    'Chronos_base', 'Chronos_small', 'Chronos_small_finetuned',
    # stacking/ensemble
    'predict_svm9', 'predict_linreg9_no_bias', 'predict_RFR'
]

# Генерация базовых временных рядов
rows = []
for article in ARTICLES:
    base = np.linspace(100, 200, N_MONTHS) + np.random.normal(0, 10, N_MONTHS)
    for i, date in enumerate(DATE_RANGE):
        row = {
            'date': date,
            'Fact': base[i],
            'Статья': article,
            'item_id': ITEM_ID,
            'factor': FACTOR
        }
        rows.append(row)
df = pd.DataFrame(rows)

# Генерация предсказаний для всех моделей
for col in MODEL_COLS:
    if col == 'NaiveModel':
        df[col] = df['Fact'] + np.random.normal(0, 8, N_MONTHS)
    elif col == 'SeasonalNaiveModel':
        df[col] = df['Fact'] + np.random.normal(0, 7, N_MONTHS)
    elif col == 'AverageModel':
        df[col] = df['Fact'] + np.random.normal(0, 6, N_MONTHS)
    elif col == 'SeasonalAverageModel':
        df[col] = df['Fact'] + np.random.normal(0, 6, N_MONTHS)
    elif col == 'ZeroModel':
        df[col] = np.zeros(N_MONTHS)
    elif col == 'AutoARIMAModel':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 6, N_MONTHS)
    elif col == 'AutoETSModel':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 5, N_MONTHS)
    elif col == 'ThetaModel':
        df[col] = df['Fact'] * np.random.uniform(0.95, 1.05, N_MONTHS) + np.random.normal(0, 7, N_MONTHS)
    elif col == 'CrostonModel':
        df[col] = df['Fact'] * np.random.uniform(0.94, 1.06, N_MONTHS) + np.random.normal(0, 7, N_MONTHS)
    elif col == 'NPTSModel':
        df[col] = df['Fact'] * np.random.uniform(0.93, 1.07, N_MONTHS) + np.random.normal(0, 8, N_MONTHS)
    elif col == 'DeepARModel':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 5, N_MONTHS)
    elif col == 'PatchTSTModel':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'TemporalFusionTransformerModel':
        df[col] = df['Fact'] * np.random.uniform(0.95, 1.05, N_MONTHS) + np.random.normal(0, 6, N_MONTHS)
    elif col == 'TiDEModel':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 5, N_MONTHS)
    elif col == 'DirectTabularModel':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'RecursiveTabularModel':
        df[col] = df['Fact'] * np.random.uniform(0.95, 1.05, N_MONTHS) + np.random.normal(0, 6, N_MONTHS)
    elif col == 'Chronos_base':
        df[col] = df['Fact'] * np.random.uniform(0.98, 1.02, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'Chronos_small':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'Chronos_small_finetuned':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'predict_svm9':
        df[col] = df['Fact'] * np.random.uniform(0.96, 1.04, N_MONTHS) + np.random.normal(0, 5, N_MONTHS)
    elif col == 'predict_linreg9_no_bias':
        df[col] = df['Fact'] * np.random.uniform(0.97, 1.03, N_MONTHS) + np.random.normal(0, 4, N_MONTHS)
    elif col == 'predict_RFR':
        df[col] = df['Fact'] * np.random.uniform(0.95, 1.05, N_MONTHS) + np.random.normal(0, 6, N_MONTHS)

# Для каждой модели считаем разницу и отклонение
for col in MODEL_COLS:
    df[f'{col}_разница'] = df[col] - df['Fact']
    df[f'{col}_отклонение_%'] = 100 * (df[col] - df['Fact']) / df['Fact']

df = df.sort_values('date').reset_index(drop=True)
df['date'] = df['date'].dt.date

# Генерация коэффициентов для линейных моделей и информации об ансамблях
coeffs_with_intercept = pd.DataFrame({
    'date': [DATE_RANGE[-1]],
    'Статья': [ARTICLES[0]],
    'predict_naive': [np.random.uniform(0.2, 0.4)],
    'predict_TS_ML': [np.random.uniform(0.3, 0.5)],
    'intercept': [np.random.uniform(10, 20)]
})

coeffs_no_intercept = pd.DataFrame({
    'date': [DATE_RANGE[-1]],
    'Статья': [ARTICLES[0]],
    'predict_naive': [np.random.uniform(0.3, 0.6)],
    'predict_TS_ML': [np.random.uniform(0.3, 0.6)]
})

ensemble_info = pd.DataFrame({
    'date': [DATE_RANGE[-1]],
    'Статья': [ARTICLES[0]],
    'model': ['Ensemble_TS_ML'],
    'weight': [1.0],
    'metric': ['MAE'],
    'score': [np.random.uniform(5, 15)]
})

# Сохранение результатов в Excel-файл
os.makedirs('results', exist_ok=True)
with pd.ExcelWriter('results/synthetic_base_pipeline.xlsx') as writer:
    df.to_excel(writer, sheet_name='data', index=False)
    coeffs_with_intercept.to_excel(writer, sheet_name='coeffs_with_intercept', index=False)
    coeffs_no_intercept.to_excel(writer, sheet_name='coeffs_no_intercept', index=False)
    ensemble_info.to_excel(writer, sheet_name='TimeSeries_ensemble_models_info', index=False)
print('Файл сохранён: results/synthetic_base_pipeline.xlsx')

Файл сохранён: results/synthetic_base_pipeline.xlsx



In [13]:
# --- Формирование файла для BASE pipeline ---
# Оставляем только нужные столбцы для листа data
base_cols = ['date', 'Fact'] + MODEL_COLS + ['Статья', 'item_id', 'factor']
df_base = df[base_cols].copy()

# Переписываем файл с правильной структурой
with pd.ExcelWriter('results/synthetic_base_pipeline_BASE.xlsx') as writer:
    df_base.to_excel(writer, sheet_name='data', index=False)
    coeffs_with_intercept.to_excel(writer, sheet_name='coeffs_with_intercept', index=False)
    coeffs_no_intercept.to_excel(writer, sheet_name='coeffs_no_intercept', index=False)
    ensemble_info.to_excel(writer, sheet_name='TimeSeries_ensemble_models_info', index=False)
print('Файл сохранён: results/synthetic_base_pipeline_BASE.xlsx')

Файл сохранён: results/synthetic_base_pipeline_BASE.xlsx



In [14]:
# --- Исправление: переименование столбца 'date' в 'Дата' для совместимости с BASE pipeline ---
df_base = df_base.rename(columns={'date': 'Дата'})
base_cols = ['Дата', 'Fact'] + MODEL_COLS + ['Статья', 'item_id', 'factor']
df_base = df_base[base_cols]

with pd.ExcelWriter('results/synthetic_base_pipeline_BASE.xlsx') as writer:
    df_base.to_excel(writer, sheet_name='data', index=False)
    coeffs_with_intercept.to_excel(writer, sheet_name='coeffs_with_intercept', index=False)
    coeffs_no_intercept.to_excel(writer, sheet_name='coeffs_no_intercept', index=False)
    ensemble_info.to_excel(writer, sheet_name='TimeSeries_ensemble_models_info', index=False)
print('Файл сохранён: results/synthetic_base_pipeline_BASE.xlsx (с "Дата")')

Файл сохранён: results/synthetic_base_pipeline_BASE.xlsx (с "Дата")


In [15]:
# --- Переименование 'date' в 'Дата' во всех листах для BASE pipeline ---
coeffs_with_intercept = coeffs_with_intercept.rename(columns={'date': 'Дата'})
coeffs_no_intercept = coeffs_no_intercept.rename(columns={'date': 'Дата'})

# Перезапись файла с правильными названиями столбцов
with pd.ExcelWriter('results/synthetic_base_pipeline_BASE.xlsx') as writer:
    df_base.to_excel(writer, sheet_name='data', index=False)
    coeffs_with_intercept.to_excel(writer, sheet_name='coeffs_with_intercept', index=False)
    coeffs_no_intercept.to_excel(writer, sheet_name='coeffs_no_intercept', index=False)
    ensemble_info.to_excel(writer, sheet_name='TimeSeries_ensemble_models_info', index=False)
print('Файл сохранён: results/synthetic_base_pipeline_BASE.xlsx (с "Дата" во всех листах)')

Файл сохранён: results/synthetic_base_pipeline_BASE.xlsx (с "Дата" во всех листах)


In [16]:
# --- Переименование 'date' в 'Дата' во всех листах для BASE pipeline (финальная версия) ---
coeffs_with_intercept = coeffs_with_intercept.rename(columns={'date': 'Дата'})
coeffs_no_intercept = coeffs_no_intercept.rename(columns={'date': 'Дата'})
ensemble_info = ensemble_info.rename(columns={'date': 'Дата'})

with pd.ExcelWriter('results/synthetic_base_pipeline_BASE.xlsx') as writer:
    df_base.to_excel(writer, sheet_name='data', index=False)
    coeffs_with_intercept.to_excel(writer, sheet_name='coeffs_with_intercept', index=False)
    coeffs_no_intercept.to_excel(writer, sheet_name='coeffs_no_intercept', index=False)
    ensemble_info.to_excel(writer, sheet_name='TimeSeries_ensemble_models_info', index=False)
print('Файл сохранён: results/synthetic_base_pipeline_BASE.xlsx (с "Дата" во всех листах)')

Файл сохранён: results/synthetic_base_pipeline_BASE.xlsx (с "Дата" во всех листах)
